In [ ]:
!pip install feedparser html2text slack_sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.8/289.8 kB 2.2 MB/s eta 0:00:00


In [ ]:
import requests
from time import sleep
# import winsound
from datetime import datetime
from datetime import timedelta
import logging
logging.basicConfig(level=logging.DEBUG)
import feedparser
import os
import html2text
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

In [ ]:
slack_token = ""
client = WebClient(token=slack_token)

cookie = ''
name = 'mod_auth_openidc_session'

def alert(msg, channel="blog-updates"):
    try:
        response = client.chat_postMessage(
            channel = channel,
            text=f"<!channel> {msg}"
        )
    except SlackApiError as e:
        # You will get a SlackApiError if "ok" is False
        assert e.response["error"]    # str like 'invalid_auth', 'channel_not_found'

def calm_alert(msg, channel="blog-updates"):
    try:
        response = client.chat_postMessage(
            channel = channel,
            text=f"{msg}"
        )
    except SlackApiError as e:
        # You will get a SlackApiError if "ok" is False
        assert e.response["error"]    # str like 'invalid_auth', 'channel_not_found'

s = requests.Session()
r1 = s.get('https://campus.placements.iitb.ac.in/blog/placement/', cookies={name: cookie})
text1 = r1.text
walk_count = text1.casefold().count('walk'.casefold())
manan_count = text1.casefold().count('deepankar sehra'.casefold())
response = client.chat_postMessage(
            channel="blog-updates",
            text=f"I am alive and overseeing the blog"
        )
i = 0
old_summary = ""
while True:

    try:
        r2 = s.get('https://campus.placements.iitb.ac.in/blog/placement/', cookies={name: cookie})
    except:
        alert(f'I AM FUCKING DEAD AT {current_time}', channel='testing')
        alert(f'I AM DEAD AT {current_time}')

    text2 = r2.text
    now = datetime.now()
    # time_change = timedelta(minutes=330)
    # now = now + time_change
    current_time = now.strftime("%H-%M-%S")

    if 'OTP' in text2:
        print(f'cookie ded at {current_time}')
        alert('The bot is down, please check manually')
        alert(f'cookie ded at {current_time}', channel='testing')
        i = 0
        sleep(300)
        continue

    if text1!=text2:
        feed = feedparser.parse(s.get('https://campus.placements.iitb.ac.in/blog/placement/?feed=rss2', cookies={name: cookie}).text)
        try:
            title = feed.entries[0]['title']
            print(title)
            summary = feed.entries[0]['content'][0]['value']
        except:
            alert("Something may be wrong. Check manually.")
            print("Something may be wrong. Check manually.")
            i = 0
            sleep(60)
            continue
        summary = html2text.html2text(summary)
        summary = summary.replace('*','')

        if summary == old_summary:
            print("same update")
            continue
        f = open(f"{current_time}.html", "w")
        f.write(text2)
        f.close()
        if text2.casefold().count('walk'.casefold()) > walk_count:
            print(f'Possible new walk-in detected at {current_time} : {title} - \n {summary}')
            alert(f'Possible new walk-in detected at {current_time} : {title} - \n {summary}')
            walk_count = text2.casefold().count('walk'.casefold())
            i = 0
        else:
            print(f"Update detected at {current_time} : {title} - \n {summary}")
            alert(f"Update detected at {current_time} : {title} - \n {summary}")
            i = 0
        text1 = text2
        old_summary = summary
    i+=1
    if i == 10:
        print(f'I am still alive at {current_time}')
        calm_alert(f'I am still alive at {current_time}', "bot-status")
        i = 0
    sleep(30)

KeyboardInterrupt: 